In [32]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

# --- Load stock returns ---
amd = pd.read_excel('AMD - Stock return.xlsx').dropna(subset=['Stock Return'])
amd['Date'] = pd.to_datetime(amd['Date'])

# --- Load Fama-French factors ---
ff = pd.read_excel('F-F_Research_Data_Factors.xlxs.xlsx', skiprows=3)
ff.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']
ff = ff.dropna(subset=['Date'])
ff['Date'] = pd.to_numeric(ff['Date'], errors='coerce')
ff = ff.dropna(subset=['Date'])
ff['Date'] = ff['Date'].astype(int).astype(str)
ff = ff[ff['Date'].str.len() == 6]
ff['Date'] = pd.to_datetime(ff['Date'], format='%Y%m')
ff[['Mkt-RF', 'SMB', 'HML', 'RF']] /= 100

# --- Merge and compute excess return ---
data = pd.merge(amd, ff, on='Date', how='inner')
data['AMD_Excess'] = data['Stock Return'] - data['RF']

for col in ['AMD_Excess', 'Mkt-RF', 'SMB', 'HML']:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data = data.dropna(subset=['AMD_Excess', 'Mkt-RF', 'SMB', 'HML'])

In [33]:
# QUESTION 1: CAPM REGRESSION
y = data['AMD_Excess']
X = sm.add_constant(data[['Mkt-RF']])
capm = sm.OLS(y, X).fit()

beta = capm.params['Mkt-RF']
alpha = capm.params['const']
r2 = capm.rsquared
alpha_pval = capm.pvalues['const']

print(capm.summary())

# Task 1: Beta
print(f"\nBeta = {beta:.4f}")

# Task 2: Alpha significance
print(f"Alpha = {alpha:.4f} (p-value = {alpha_pval:.4f})")
print("Significant at 5%" if alpha_pval < 0.05 else "Not significant at 5%")

# Task 3: R-squared
print(f"R² = {r2:.4f} ({r2*100:.1f}% of variance explained by market)")

# Task 4: Expected excess return if market = 10%
expected = alpha + beta * 0.10
print(f"Expected excess return = {expected:.4f}")

                            OLS Regression Results                            
Dep. Variable:             AMD_Excess   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     22.86
Date:                Wed, 18 Feb 2026   Prob (F-statistic):           2.07e-05
Time:                        00:40:53   Log-Likelihood:                 30.184
No. Observations:                  45   AIC:                            -56.37
Df Residuals:                      43   BIC:                            -52.75
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0047      0.019      0.247      0.8

In [34]:
# QUESTION 2: COMPARING TWO STOCKS (AMD vs Berkshire Hathaway)

# Load Berkshire data
berk = pd.read_excel("Berkshire Hathaway Inc.xlsx")
berk['Date'] = pd.to_datetime(berk['Date'])
berk['Stock_Return'] = berk.iloc[:, 5].pct_change()
berk = berk.dropna(subset=['Stock_Return'])

# Normalize dates to first of month for matching
berk['Date'] = berk['Date'].to_numpy().astype('datetime64[M]')

# Merge with FF
m_berk = pd.merge(berk[['Date', 'Stock_Return']], ff[['Date', 'Mkt-RF', 'RF']],
                   on='Date', how='inner')
m_berk['Excess'] = m_berk['Stock_Return'] - m_berk['RF']

# Force numeric
for col in ['Excess', 'Mkt-RF', 'RF']:
    m_berk[col] = pd.to_numeric(m_berk[col], errors='coerce')
m_berk = m_berk.dropna(subset=['Excess', 'Mkt-RF'])

# Task 1: CAPM for Berkshire
X_berk = sm.add_constant(m_berk['Mkt-RF'])
capm_berk = sm.OLS(m_berk['Excess'], X_berk).fit()

print("AMD:")
print(capm.summary())
print("\nBERKSHIRE:")
print(capm_berk.summary())

print(f"\nAMD Beta: {capm.params['Mkt-RF']:.4f}")
print(f"Berkshire Beta: {capm_berk.params['Mkt-RF']:.4f}")

# Task 2: F-test for equal betas
Y_pool = np.concatenate([data['AMD_Excess'].values, m_berk['Excess'].values])
X_pool = np.concatenate([data['Mkt-RF'].values, m_berk['Mkt-RF'].values])
D = np.concatenate([np.ones(len(data)), np.zeros(len(m_berk))])
N = len(Y_pool)

X_r = sm.add_constant(np.column_stack([X_pool, D]))
X_ur = sm.add_constant(np.column_stack([X_pool, D, D * X_pool]))

m_r = sm.OLS(Y_pool, X_r).fit()
m_ur = sm.OLS(Y_pool, X_ur).fit()

f_stat = ((m_r.ssr - m_ur.ssr) / 1) / (m_ur.ssr / (N - 4))
p_val = 1 - stats.f.cdf(f_stat, 1, N - 4)

print(f"\nF-test for equal betas: F = {f_stat:.4f}, p = {p_val:.4f}")
print("Reject H0: betas are different" if p_val < 0.05 else "Fail to reject H0")

AMD:
                            OLS Regression Results                            
Dep. Variable:             AMD_Excess   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     22.86
Date:                Wed, 18 Feb 2026   Prob (F-statistic):           2.07e-05
Time:                        00:44:50   Log-Likelihood:                 30.184
No. Observations:                  45   AIC:                            -56.37
Df Residuals:                      43   BIC:                            -52.75
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0047      0.019      0.247    

In [33]:
# QUESTION 3: FAMA-FRENCH 3-FACTOR MODEL
y = data['AMD_Excess']
X_ff3 = sm.add_constant(data[['Mkt-RF', 'SMB', 'HML']])
ff3 = sm.OLS(y, X_ff3).fit()

print(ff3.summary())

# Task 1: R² comparison
print(f"\nCAPM R²: {r2:.4f} | FF3 R²: {ff3.rsquared:.4f}")
print(f"Improvement: {(ff3.rsquared - r2)*100:.2f}%")

# Task 2: F-test for joint significance of SMB and HML
ssr_r = capm.ssr   # restricted model (CAPM)
ssr_u = ff3.ssr    # unrestricted model (FF3)
n = len(y)
q = 2
k = 4
f_stat = ((ssr_r - ssr_u) / q) / (ssr_u / (n - k))
p_val = 1 - stats.f.cdf(f_stat, q, n - k)
print(f"\nF-test: F = {f_stat:.4f}, p-value = {p_val:.4f}")

# Task 3: SMB vs HML sensitivity
print(f"\nSMB coeff: {ff3.params['SMB']:.4f} (p = {ff3.pvalues['SMB']:.4f})")
print(f"HML coeff: {ff3.params['HML']:.4f} (p = {ff3.pvalues['HML']:.4f})")

# Task 4: Alpha comparison
print(f"\nCAPM Alpha: {alpha:.4f} (p = {alpha_pval:.4f})")
print(f"FF3 Alpha:  {ff3.params['const']:.4f} (p = {ff3.pvalues['const']:.4f})")

                            OLS Regression Results                            
Dep. Variable:             AMD_Excess   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.366
Method:                 Least Squares   F-statistic:                     9.459
Date:                Tue, 17 Feb 2026   Prob (F-statistic):           7.13e-05
Time:                        16:38:40   Log-Likelihood:                 32.427
No. Observations:                  45   AIC:                            -56.85
Df Residuals:                      41   BIC:                            -49.63
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0065      0.019      0.334      0.7

In [34]:
# QUESTION 4: INDUSTRY PORTFOLIO ANALYSIS

# Load industry data
industries = pd.read_excel('5_Industry_Portfolios.xlsx', sheet_name='VW Monthly', skiprows=2)
ind = industries[['Date', 'HiTec', 'Hlth']].copy()
ind.columns = ['Date', 'Tech', 'Health']
ind['Date'] = pd.to_numeric(ind['Date'], errors='coerce')
ind = ind.dropna(subset=['Date'])
ind['Date'] = pd.to_datetime(ind['Date'].astype(int).astype(str), format='%Y%m')
ind = ind.set_index('Date')

# Merge with FF data
ff_q4 = ff.set_index('Date')[['Mkt-RF', 'SMB', 'HML', 'RF']]
ind_data = ind.join(ff_q4, how='inner')
ind_data[['Tech', 'Health']] /= 100
ind_data['Tech_ex'] = ind_data['Tech'] - ind_data['RF']
ind_data['Health_ex'] = ind_data['Health'] - ind_data['RF']

for col in ['Tech_ex', 'Health_ex', 'Mkt-RF', 'SMB', 'HML']:
    ind_data[col] = pd.to_numeric(ind_data[col], errors='coerce')
ind_data = ind_data.dropna(subset=['Tech_ex', 'Health_ex', 'Mkt-RF', 'SMB', 'HML'])

# Run FF3 regressions
X = sm.add_constant(ind_data[['Mkt-RF', 'SMB', 'HML']])
tech_model = sm.OLS(ind_data['Tech_ex'], X).fit()
health_model = sm.OLS(ind_data['Health_ex'], X).fit()

print("TECHNOLOGY:")
print(tech_model.summary())
print("\nHEALTHCARE:")
print(health_model.summary())

# Task 1: Market beta comparison
print(f"\nTech beta: {tech_model.params['Mkt-RF']:.4f}")
print(f"Health beta: {health_model.params['Mkt-RF']:.4f}")

# Task 2: SMB exposure
print(f"\nTech SMB: {tech_model.params['SMB']:.4f} (p = {tech_model.pvalues['SMB']:.4f})")
print(f"Health SMB: {health_model.params['SMB']:.4f} (p = {health_model.pvalues['SMB']:.4f})")

# Task 3: F-test for equal market betas
t = ind_data[['Tech_ex', 'Mkt-RF', 'SMB', 'HML']].rename(columns={'Tech_ex': 'R'})
h = ind_data[['Health_ex', 'Mkt-RF', 'SMB', 'HML']].rename(columns={'Health_ex': 'R'})
t['D'] = 1
h['D'] = 0
pooled = pd.concat([t, h]).dropna()
pooled['D_Mkt'] = pooled['D'] * pooled['Mkt-RF']

m_r = sm.OLS(pooled['R'], sm.add_constant(pooled[['D', 'Mkt-RF', 'SMB', 'HML']])).fit()
m_ur = sm.OLS(pooled['R'], sm.add_constant(pooled[['D', 'Mkt-RF', 'SMB', 'HML', 'D_Mkt']])).fit()

f_stat = ((m_r.ssr - m_ur.ssr) / 1) / (m_ur.ssr / (len(pooled) - 6))
p_val = 1 - stats.f.cdf(f_stat, 1, len(pooled) - 6)
print(f"\nF-test for equal betas: F = {f_stat:.4f}, p = {p_val:.4f}")

# Task 4: Recession strategy
tech_drop = tech_model.params['const'] + tech_model.params['Mkt-RF'] * (-0.10)
health_drop = health_model.params['const'] + health_model.params['Mkt-RF'] * (-0.10)
print(f"\nIf market drops 10%: Tech = {tech_drop:.4f}, Health = {health_drop:.4f}")

TECHNOLOGY:
                            OLS Regression Results                            
Dep. Variable:                Tech_ex   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     2409.
Date:                Tue, 17 Feb 2026   Prob (F-statistic):               0.00
Time:                        16:38:44   Log-Likelihood:                 2896.1
No. Observations:                1183   AIC:                            -5784.
Df Residuals:                    1179   BIC:                            -5764.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0011      0.001     -1.

In [35]:
# QUESTION 5: FAMA-FRENCH FIVE-FACTOR MODEL
import statsmodels.formula.api as smf
# load stock returns (same as Q1)
amd = pd.read_excel('AMD - Stock return.xlsx').dropna(subset=['Stock Return'])
amd['Date'] = pd.to_datetime(amd['Date'])

# load 5-factor data (header is row 5 in Excel = skiprows=4)
ff5 = pd.read_excel('F-F_Research_Data_5_Factors_2x3.xlsx', skiprows=4)
ff5.columns = [c.strip() for c in ff5.columns]
ff5.rename(columns={ff5.columns[0]: 'Date'}, inplace=True)
ff5['Date'] = pd.to_numeric(ff5['Date'], errors='coerce')
ff5 = ff5.dropna(subset=['Date'])
ff5['Date'] = ff5['Date'].astype(int).astype(str)
ff5 = ff5[ff5['Date'].str.len() == 6]
ff5['Date'] = pd.to_datetime(ff5['Date'], format='%Y%m')
for c in ['Mkt-RF','SMB','HML','RMW','CMA','RF']:
    ff5[c] = pd.to_numeric(ff5[c], errors='coerce') / 100


# merge
df5 = pd.merge(amd, ff5, on='Date', how='inner').dropna()
df5['AMD_Excess'] = df5['Stock Return'] - df5['RF']
print(f"Observations: {len(df5)}")

# run all three models
capm = smf.ols("AMD_Excess ~ Q('Mkt-RF')", data=df5).fit()
model_3F = smf.ols("AMD_Excess ~ Q('Mkt-RF') + SMB + HML", data=df5).fit()
model_5F = smf.ols("AMD_Excess ~ Q('Mkt-RF') + SMB + HML + RMW + CMA", data=df5).fit()

print(capm.summary())
print(model_3F.summary())
print(model_5F.summary())

# task 1: test if all five factors are jointly significant
# H0: all betas = 0
restricted = smf.ols("AMD_Excess ~ 1", data=df5).fit()
F_all = model_5F.compare_f_test(restricted)

r2 = model_5F.rsquared
k, n = 5, len(df5)
F_manual = (r2 / k) / ((1 - r2) / (n - k - 1))

print(f"\nTest 1 - joint significance of all factors:")
print(f"  F-stat (Python):  {F_all[0]:.4f},  p = {F_all[1]:.4f}")
print(f"  F-stat (manual):  {F_manual:.4f}")

# task 2: do RMW and CMA add anything beyond the 3-factor model?
# H0: beta_RMW = beta_CMA = 0
F_incr = model_5F.compare_f_test(model_3F)
print(f"\nTest 2 - RMW & CMA incremental test:")
print(f"  F-stat: {F_incr[0]:.4f},  p = {F_incr[1]:.4f}")

# task 3: test if SMB and HML coefficients are equal
# H0: beta_SMB = beta_HML
F_equal = model_5F.f_test("SMB = HML")
print(f"\nTest 3 - SMB = HML:")
print(F_equal)

# task 4: model comparison
print(f"\nModel Comparison:")
print(f"  CAPM     - Adj R2: {capm.rsquared_adj:.5f},  AIC: {capm.aic:.2f}")
print(f"  3-Factor - Adj R2: {model_3F.rsquared_adj:.5f},  AIC: {model_3F.aic:.2f}")
print(f"  5-Factor - Adj R2: {model_5F.rsquared_adj:.5f},  AIC: {model_5F.aic:.2f}")

Observations: 57
                            OLS Regression Results                            
Dep. Variable:             AMD_Excess   R-squared:                       0.277
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                     21.11
Date:                Wed, 18 Feb 2026   Prob (F-statistic):           2.58e-05
Time:                        00:45:41   Log-Likelihood:                 30.734
No. Observations:                  57   AIC:                            -57.47
Df Residuals:                      55   BIC:                            -53.38
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0120      0.019

In [36]:
# QUESTION 6: LONG-TERM REVERSAL EFFECT

# Load reversal portfolios
rev = pd.read_csv("10_Portfolios_Prior_60_13.CSV", skiprows=10, header=0)
rev.rename(columns={'Unnamed: 0': 'YM'}, inplace=True)
rev['YM'] = rev['YM'].astype(str).str.strip()
rev = rev[rev['YM'].str.match(r'^\d{6}$')]
rev = rev.apply(lambda c: pd.to_numeric(c, errors='coerce')).replace([-99.99, -999], np.nan)
# Keep only first section (value-weighted) — file has VW then EW stacked
rev = rev.iloc[:len(rev)//2]
rev.columns = ['YM','P1','P2','P3','P4','P5','P6','P7','P8','P9','P10']
rev['YM'] = rev['YM'].astype(int).astype(str)


# Load FF factors
ff_q6 = pd.read_csv("F-F_Research_Data_Factors.CSV", skiprows=3, header=0, index_col=0)
ff_q6.index = ff_q6.index.astype(str).str.strip()
ff_q6 = ff_q6[ff_q6.index.str.match(r'^\d{6}$')]
ff_q6 = ff_q6.apply(lambda c: pd.to_numeric(c, errors='coerce'))
ff_q6.columns = [c.strip() for c in ff_q6.columns]
ff_q6['YM'] = ff_q6.index.str.strip()

# Merge on year-month
df_q6 = pd.merge(rev, ff_q6[['YM','Mkt-RF','RF']], on='YM', how='inner').dropna().reset_index(drop=True)


loser = df_q6['P1'].values
winner = df_q6['P10'].values
reversal = loser - winner

# Task 1: average monthly returns
print(f"\nLoser (P1) avg return: {loser.mean():.4f}%")
print(f"Winner (P10) avg return: {winner.mean():.4f}%")

# Task 2: reversal strategy return
print(f"Long-Short avg return: {reversal.mean():.4f}% per month")

# Task 3: t-test on reversal premium
n = len(reversal)
mean_r = reversal.mean()
se_r = reversal.std(ddof=1) / np.sqrt(n)
t_stat = mean_r / se_r
p_val = 2 * (1 - stats.t.cdf(abs(t_stat), df=n-1))
print(f"\nt-test: t={t_stat:.4f}, p={p_val:.4f}")
print(f"Mean: {mean_r:.4f}%, SE: {se_r:.4f}%")

# Task 4: compare 20th vs 21st century
def ttest(series):
    m = series.mean()
    se = series.std(ddof=1) / np.sqrt(len(series))
    t = m / se
    p = 2 * (1 - stats.t.cdf(abs(t), df=len(series)-1))
    return m, t, p

years = df_q6['YM'].str[:4].astype(int).values
m20, t20, p20 = ttest(reversal[years < 2000])
m21, t21, p21 = ttest(reversal[years >= 2000])
print(f"\n20th century: mean={m20:.4f}%, t={t20:.4f}, p={p20:.4f}")
print(f"21st century: mean={m21:.4f}%, t={t21:.4f}, p={p21:.4f}")

# Task 5: CAPM regression on the reversal portfolio
y = reversal - df_q6['RF'].values
x = df_q6['Mkt-RF'].values

good = np.isfinite(y) & np.isfinite(x)
y, x = y[good], x[good]

X = np.column_stack([np.ones(len(x)), x])
b = np.linalg.lstsq(X, y, rcond=None)[0]
alpha_rev, beta_rev = b

resid = y - X @ b
mse = np.sum(resid**2) / (len(y) - 2)
cov_mat = mse * np.linalg.pinv(X.T @ X)

t_a = alpha_rev / np.sqrt(cov_mat[0,0])
t_b = beta_rev / np.sqrt(cov_mat[1,1])
p_a = 2 * (1 - stats.t.cdf(abs(t_a), df=len(y)-2))
p_b = 2 * (1 - stats.t.cdf(abs(t_b), df=len(y)-2))
r2_rev = 1 - np.sum(resid**2) / np.sum((y - y.mean())**2)

print(f"\nCAPM: Alpha={alpha_rev:.4f}% (t={t_a:.4f}, p={p_a:.4f})")
print(f"      Beta={beta_rev:.4f} (t={t_b:.4f}, p={p_b:.4f})")
print(f"      R2={r2_rev:.4f}")


Loser (P1) avg return: 1.6886%
Winner (P10) avg return: 0.9468%
Long-Short avg return: 0.7418% per month

t-test: t=4.9606, p=0.0000
Mean: 0.7418%, SE: 0.1495%

20th century: mean=0.9623%, t=5.1923, p=0.0000
21st century: mean=0.1333%, t=0.5749, p=0.5656

CAPM: Alpha=0.2628% (t=1.7820, p=0.0749)
      Beta=0.2984 (t=10.7737, p=0.0000)
      R2=0.0490


In [37]:
# QUESTION 7: IS THE VALUE PREMIUM DEAD?

# load book-to-market sorted portfolios
port = pd.read_excel("100_Portfolios_10x10.xlsx", header=15, index_col=0)
port.index = port.index.astype(str).str.strip()
port = port[port.index.str.fullmatch(r"\d{6}")]

# drop unnamed columns and clean
port = port[[c for c in port.columns if 'Unnamed' not in str(c)]]
port = port.apply(pd.to_numeric, errors='coerce').replace([-99.99, -999], np.nan)
port = port.dropna(how='all') / 100
port.index = pd.to_datetime(port.index, format='%Y%m')


# Growth = first col (low B/M), Value = last col (high B/M)
growth = port.iloc[:, 0]
value = port.iloc[:, -1]

# task 1: average monthly returns
print(f"\nTask 1:")
print(f"  Growth avg return: {growth.mean():.5f}")
print(f"  Value avg return:  {value.mean():.5f}")
print(f"  Growth = low book-to-market (growth stocks)")
print(f"  Value = high book-to-market (value stocks)")

# task 2: long-short portfolio (long value, short growth)
value_premium = value - growth

print(f"\nTask 2:")
print(f"  Value premium (long value, short growth): {value_premium.mean():.5f} per month")

# task 3: t-tests by century
years = port.index.year

vp_20c = value_premium[(years >= 1963) & (years <= 1999)].dropna()
vp_21c = value_premium[years >= 2000].dropna()

t20 = stats.ttest_1samp(vp_20c, 0)
t21 = stats.ttest_1samp(vp_21c, 0)

print(f"\nTask 3:")
print(f"  20th century: mean={vp_20c.mean():.5f}, t={t20.statistic:.3f}, p={t20.pvalue:.5f}")
print(f"  21st century: mean={vp_21c.mean():.5f}, t={t21.statistic:.3f}, p={t21.pvalue:.5f}")

# task 4: is the value premium different between centuries?
diff_test = stats.ttest_ind(vp_20c, vp_21c, equal_var=False)

# manual calculation
m1, m2 = vp_20c.mean(), vp_21c.mean()
v1, v2 = vp_20c.var(ddof=1), vp_21c.var(ddof=1)
n1, n2 = len(vp_20c), len(vp_21c)
t_manual = (m1 - m2) / np.sqrt(v1/n1 + v2/n2)

print(f"\nTask 4:")
print(f"  Difference in means t-stat (Python): {diff_test.statistic:.3f}, p={diff_test.pvalue:.5f}")
print(f"  Difference in means t-stat (manual): {t_manual:.3f}")


Task 1:
  Growth avg return: 0.00949
  Value avg return:  0.01096
  Growth = low book-to-market (growth stocks)
  Value = high book-to-market (value stocks)

Task 2:
  Value premium (long value, short growth): 0.00007 per month

Task 3:
  20th century: mean=0.00132, t=0.407, p=0.68404
  21st century: mean=-0.00197, t=-0.470, p=0.63888

Task 4:
  Difference in means t-stat (Python): 0.621, p=0.53497
  Difference in means t-stat (manual): 0.621
